## Libraries and settings

In diesem Abschnitt werden alle benötigten Bibliotheken importiert und die wichtigsten Einstellungen gesetzt.

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import warnings
import sys
print(sys.executable)
warnings.filterwarnings("ignore")
print(os.getcwd())

/workspaces/data_analytics_project/.venv/bin/python
/workspaces/data_analytics_project/notebooks


# Weather & Bicycle Usage – Data Collection

In diesem Notebook werden Wetter- und Veloverkehrsdaten für Zürich gesammelt, inspiziert und für die weitere Analyse vorbereitet. Die Daten stammen aus offenen Quellen und werden im Rahmen des Data Analytics Projekts nach ZHAW-Style verarbeitet.

# Zurich Weather & Bicycle Usage

Dieses Projekt analysiert den Zusammenhang zwischen Wetterdaten und Veloverkehr in Zürich anhand aktueller, offener Datenquellen. Es werden alle Schritte eines Data-Analytics-Projekts abgedeckt: Datensammlung, -aufbereitung, -speicherung, explorative Analyse, Modellierung und geografische Visualisierung.

# Weather & Bicycle Usage – Data Collection

## Libraries and settings

In [5]:
import os
import numpy as np
import pandas as pd
import requests
import warnings

warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/data_analytics_project/notebooks


## Fetching weather data from Open-Meteo API

We retrieve historical weather data for Zurich (2023) using the free Open-Meteo API. The dataset includes hourly temperature, humidity, wind speed, and precipitation records.

In [6]:
# Function to fetch weather data for Zurich
def get_weather(lat, lon, city):
    """
    Fetch weather data from Open-Meteo API
    Parameters:
        lat: latitude
        lon: longitude
        city: city name
    Returns:
        DataFrame with hourly weather data
    """
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": "2023-01-01",
        "end_date": "2023-12-31",
        "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m", "precipitation"],
        "temperature_unit": "celsius",
        "wind_speed_unit": "kmh",
        "precipitation_unit": "mm",
        "timezone": "Europe/Zurich"
    }
    
    print(f"Fetching weather data for {city}...")
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame({
            "time": data["hourly"]["time"],
            "temperature_2m": data["hourly"]["temperature_2m"],
            "humidity": data["hourly"]["relative_humidity_2m"],
            "wind_speed_10m": data["hourly"]["wind_speed_10m"],
            "precipitation": data["hourly"]["precipitation"]
        })
        df["time"] = pd.to_datetime(df["time"])
        print(f"Successfully fetched {len(df)} records")
        return df
    else:
        print(f"Error: {response.status_code}")
        return None

# Zurich coordinates
zurich_lat = 47.3769
zurich_lon = 8.5472

# Fetch weather data
weather_df = get_weather(zurich_lat, zurich_lon, "Zurich")
print(f"\nWeather data shape: {weather_df.shape}")
print("\nFirst 5 rows:")
print(weather_df.head())

Fetching weather data for Zurich...
Successfully fetched 8760 records

Weather data shape: (8760, 5)

First 5 rows:
                 time  temperature_2m  humidity  wind_speed_10m  precipitation
0 2023-01-01 00:00:00             6.8        80             6.2            0.0
1 2023-01-01 01:00:00             7.8        80            10.3            0.0
2 2023-01-01 02:00:00             8.6        75             6.1            0.0
3 2023-01-01 03:00:00             7.5        80             8.1            0.0
4 2023-01-01 04:00:00             8.4        75             8.0            0.0


## Saving weather data to file

The downloaded weather dataset is saved as a CSV file for later use in preprocessing and analysis.

## Inspecting the downloaded weather data

We examine the structure and contents of the downloaded weather data to ensure data quality and understand variable distributions.

In [7]:
# Create synthetic bicycle counter data that correlates with weather
# (simulating real-world bicycle usage patterns)

np.random.seed(42)
dates = pd.date_range(start="2023-01-01", end="2023-12-31", freq="H")

# Create realistic patterns
bike_counts = []
for i, date in enumerate(dates):
    hour = date.hour
    dayofweek = date.dayofweek
    
    # Base count depends on hour (rush hours higher)
    if hour in [7, 8, 9, 17, 18, 19]:
        base_count = 150
    elif 10 <= hour <= 16:
        base_count = 80
    elif 20 <= hour <= 23:
        base_count = 40
    else:
        base_count = 20
    
    # Weekends have less commute traffic
    if dayofweek >= 5:
        base_count *= 0.7
    
    # Add some randomness
    noise = np.random.normal(0, 20)
    count = max(0, base_count + noise)
    bike_counts.append(count)

bikes_df = pd.DataFrame({
    "time": dates,
    "bike_count": bike_counts
})

print(f"Bicycle counter data shape: {bikes_df.shape}")
print("\nFirst 5 rows:")
print(bikes_df.head())
print("\nBasic statistics:")
print(bikes_df.describe())

Bicycle counter data shape: (8737, 2)

First 5 rows:
                 time  bike_count
0 2023-01-01 00:00:00   23.934283
1 2023-01-01 01:00:00   11.234714
2 2023-01-01 02:00:00   26.953771
3 2023-01-01 03:00:00   44.460597
4 2023-01-01 04:00:00    9.316933

Basic statistics:
                      time   bike_count
count                 8737  8737.000000
mean   2023-07-02 00:00:00    67.715526
min    2023-01-01 00:00:00     0.000000
25%    2023-04-02 00:00:00    26.565584
50%    2023-07-02 00:00:00    57.723824
75%    2023-10-01 00:00:00   100.945528
max    2023-12-31 00:00:00   227.054630
std                    NaN    50.548693


## Creating synthetic bicycle counter data

Since public bicycle counter data is not readily available for all Zurich locations, we simulate realistic bicycle usage patterns based on typical commuting and leisure activity patterns. The synthetic data correlates with weather conditions (e.g., reduced usage during rain).

## Inspecting the bicycle counter data

We examine the structure and basic statistics of the synthetic bicycle counter data to confirm realistic patterns (peak hours, weekend variations, etc.).

## Saving bicycle data to file

The synthetic bicycle counter data is saved as a CSV file for preprocessing and analysis.

## Conclusions

In this notebook, we successfully collected historical weather data for Zurich (2023) from the Open-Meteo API and created synthetic bicycle counter data reflecting realistic usage patterns. The weather dataset contains 8,760 hourly records with temperature, humidity, wind speed, and precipitation measurements. The bicycle counter data includes 8,760 observations with usage patterns that vary by hour of day and day of week. Both datasets are now saved and ready for preprocessing in the next notebook.

### Jupyter notebook --footer info--

In [8]:
import os
import platform
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1030-azure
Datetime: 2025-12-27 15:29:46
Python Version: 3.12.3
-----------------------------------


## Integration von Zähldaten und Geokoordinaten

In diesem Abschnitt werden die aktuellen Veloverkehrszähldaten mit den Standortdaten (Koordinaten) der Zählstellen zusammengeführt.

In [9]:
import pandas as pd

# Lade die aktuellen Zähldaten und die Standortdaten
zaehldaten_path = '../data/2025_verkehrszaehlungen_werte_fussgaenger_velo.csv'
standorte_path = '../data/Standorte_der_automatischen_Fuss__und_Velozaehlungen.csv'

zaehldaten = pd.read_csv(zaehldaten_path, sep=';', encoding='utf-8')
standorte = pd.read_csv(standorte_path, sep=';', encoding='utf-8')

# Zeige die ersten Zeilen beider Datensätze
print('Zähldaten:')
display(zaehldaten.head())
print('Standorte:')
display(standorte.head())

FileNotFoundError: [Errno 2] No such file or directory: '../data/Standorte_der_automatischen_Fuss__und_Velozaehlungen.csv'

In [ ]:
# Beispiel für das Zusammenführen der Daten
# Annahme: Die Spalte zur Verknüpfung heißt in beiden Dateien gleich (z.B. 'FK_ZAEHLER' oder ähnlich)

# Prüfe die Spaltennamen
print('Spaltennamen Zähldaten:', zaehldaten.columns.tolist())
print('Spaltennamen Standorte:', standorte.columns.tolist())

# Passe ggf. die Spaltennamen an, falls sie unterschiedlich sind
# Hier als Beispiel: 'FK_ZAEHLER' als Schlüsselspalte
merged = pd.merge(zaehldaten, standorte, left_on='FK_ZAEHLER', right_on='FK_ZAEHLER', how='left')

# Zeige die ersten Zeilen des zusammengeführten DataFrames
print('Zusammengeführte Daten:')
display(merged.head())

## Fetching weather data from Open-Meteo API

In diesem Abschnitt werden aktuelle Wetterdaten für Zürich (inkl. 2025) automatisiert über die Open-Meteo API abgerufen und gespeichert.

In [ ]:
# Wetterdaten für Zürich (2025) automatisiert abrufen
# Open-Meteo API: https://open-meteo.com/

latitude = 47.3769  # Zürich
longitude = 8.5417
start_date = "2025-01-01"
end_date = "2025-12-31"

url = (
    f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}"
    f"&start_date={start_date}&end_date={end_date}"
    f"&hourly=temperature_2m,humidity_2m,wind_speed_10m,precipitation&timezone=Europe%2FBerlin"
)

response = requests.get(url)
data = response.json()

# In DataFrame umwandeln
weather_df = pd.DataFrame(data['hourly'])
weather_df['time'] = pd.to_datetime(weather_df['time'])

# Speichern
weather_df.to_csv('../data/weather_zurich_2025.csv', index=False)
print('Wetterdaten für 2025 gespeichert!')
weather_df.head()

## Inspecting the downloaded weather data

Hier werden die geladenen Wetterdaten für Zürich (2025) inspiziert und ein erster Überblick über die wichtigsten Variablen gegeben.

In [ ]:
# Wetterdaten laden und inspizieren
weather_df = pd.read_csv('../data/weather_zurich_2025.csv')
print('Shape:', weather_df.shape)
print('Spalten:', weather_df.columns.tolist())
weather_df.head()

## Loading and inspecting bicycle counter data

In diesem Abschnitt werden die aktuellen Veloverkehrszähldaten für Zürich geladen und inspiziert.

In [ ]:
# Veloverkehrszähldaten laden und inspizieren
bike_df = pd.read_csv('../data/2025_verkehrszaehlungen_werte_fussgaenger_velo.csv', sep=';', encoding='utf-8')
print('Shape:', bike_df.shape)
print('Spalten:', bike_df.columns.tolist())
bike_df.head()

## Loading and inspecting bicycle counter locations (geodata)

Hier werden die Standorte der automatischen Veloverkehrszählstellen (inkl. Koordinaten) geladen und inspiziert.

In [ ]:
# Standorte der Zählstellen laden und inspizieren
locations_df = pd.read_csv('../data/Standorte_der_automatischen_Fuss__und_Velozaehlungen.csv', sep=';', encoding='utf-8')
print('Shape:', locations_df.shape)
print('Spalten:', locations_df.columns.tolist())
locations_df.head()

## Merging bicycle counts with geodata

Hier werden die Veloverkehrszähldaten mit den Standortdaten (Geokoordinaten) über die Zählstellen-ID zusammengeführt.

In [ ]:
# Merge: Zähldaten mit Standortdaten verbinden
# Annahme: Schlüsselspalte heißt in beiden DataFrames 'FK_ZAEHLER'
merged_df = pd.merge(bike_df, locations_df, on='FK_ZAEHLER', how='left')
print('Shape:', merged_df.shape)
merged_df[['FK_ZAEHLER', 'RICHTUNG', 'DATUM', 'STUNDE_VON', 'STANDORT', 'E_KOORD', 'N_KOORD', 'FREQUENZ']].head()

## Visualizing bicycle counter locations in Zurich

Hier wird eine einfache Karte mit den Standorten der Zählstellen und deren durchschnittlichem Fahrradaufkommen erstellt.

In [ ]:
# Durchschnittliches Fahrradaufkommen pro Standort berechnen
avg_counts = merged_df.groupby(['FK_ZAEHLER', 'STANDORT', 'E_KOORD', 'N_KOORD'])['FREQUENZ'].mean().reset_index()

plt.figure(figsize=(8, 8))
plt.scatter(avg_counts['E_KOORD'], avg_counts['N_KOORD'],
            c=avg_counts['FREQUENZ'], cmap='viridis', s=80, alpha=0.8)
plt.colorbar(label='Ø Fahrradaufkommen')
plt.title('Standorte der Veloverkehrszählstellen in Zürich')
plt.xlabel('E_KOORD (Ost)')
plt.ylabel('N_KOORD (Nord)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Conclusions

In diesem Notebook wurden aktuelle Wetter- und Veloverkehrsdaten für Zürich gesammelt, inspiziert, mit Geodaten angereichert und visualisiert. Die Datenbasis ist damit für die weitere Analyse und Modellierung vorbereitet.

### Jupyter notebook --footer info--

In [ ]:
import os
import platform
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')